<h1><center><font size =7 color="#B3172D">TAB MORA</font> </center></h1>

In [274]:
# Librerias
import pandas as pd #dataframe
import os #direcciones
import numpy as np #formato de variables la importar
import glob# leer archivos con el mismo nombre de una ruta especifica
import matplotlib.pyplot as plt # Graficar
import seaborn as sns 
pd.set_option('display.max_rows', None) 
import datetime as dt
from matplotlib.pyplot import figure
from seaborn import distplot 
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.width', -1)
pd.set_option('display.max_columns',None)
from datetime import datetime
from dateutil.parser import parse
import ctypes
import dateutil.relativedelta

In [275]:
%load_ext jupyternotify

The jupyternotify extension is already loaded. To reload it, use:
  %reload_ext jupyternotify


In [276]:
import time
start = time.time()

%load_ext jupyternotify #Comentar esta línea sí no se tiene instalada la libreria.

In [277]:
def Mbox(title, text, style):
    return ctypes.windll.user32.MessageBoxW(0, text, title, style)

###### Devuelve un mensaje cuando termina la ejecución del programa
%%notify
import time
time.sleep(5)

<h1><font size =7 color="#094C73">Code starts from here</font> </h1>

In [278]:
#Historia mensual No Homologado
# path_input=(r'\\192.168.0.16\Gerencia de Riesgos\SARC de GESTION\MODELOS_SARC_GESTION\ENTRADA_BEHAVESCORE\HISTORIA_MENSUAL')
# os.chdir(r'\\192.168.0.16\Gerencia de Riesgos\SARC de GESTION\MODELOS_SARC_GESTION\ENTRADA_BEHAVESCORE\HISTORIA_MENSUAL')

In [279]:
#Ruta
path_input=(r'\Users\migue\Python\behave_score\HISTORIA_MENSUAL')
os.chdir(r'\Users\migue\Python\behave_score\HISTORIA_MENSUAL')
filenames=glob.glob('**/Mora**.txt', recursive=True)

In [280]:
#Población objetivo mora1
pob_objetivo = input('Ingrese la fecha de mora1 "AAAAMM": ')
year = pob_objetivo[:4]
month = pob_objetivo[4:6]
date_pob = (year + '-' + month)
my_date_pob = datetime.strptime(date_pob, "%Y-%m")
file_name_mora1 = ('Mora_' + pob_objetivo + '.txt')

Ingrese la fecha de mora1 "AAAAMM": 201506


In [281]:
if file_name_mora1 in filenames:
    mora1= pd.read_csv(file_name_mora1, sep=None, engine='python')
    mora1.columns = mora1.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('@', '').str.replace('.', '').str.replace('á','a').str.replace('é','e').str.replace('í','i').str.replace('ó','o').str.replace('ú','u').str.replace(',','.')
else:
    Mbox('Error', 'El archivo ' + file_name_mora1 + ' no existe en el directorio actual:' + '\n' + str(path_input), 0x1000)

In [282]:
mora1_copy = mora1.copy()
mora1_copy = mora1_copy.drop(mora1_copy.columns[[4,0]], axis=1)
tab_mora = mora1_copy.copy()
tab_mora.dropna(thresh=1000 ,axis=1, inplace=True)
tab_mora.columns = (['id', 'obligacion', 'cod_credito', 'mora1'])
tab_mora.head()

,id,obligacion,cod_credito,mora1
0,74300362,200000252372,2001,0
1,87433209,200000019555,2001,1
2,19115492,200000467714,2001,0
3,30971295,200000014085,2001,0
4,19344384,200000099019,2001,0


###### Cantidad de meses => cant_mes_tab = 36

In [283]:
#Pide al usuario la cantidad de meses para construir el vector
# cant_mes_tab = input('Ingrese la cantidad de meses hacia atrás para la construcción del TAB_MORA: ')
cant_mes_tab =  17
lista = []
date=[]
for i in np.arange(1,int(cant_mes_tab)+1,1):
    lista.append(my_date_pob + dateutil.relativedelta.relativedelta(months=-int(i))) 
for item in lista:
    date.append('Mora_' + item.strftime('%Y%m') + '.txt')

In [284]:
#Archivos cargados
print('Archivos proximos a cargar:')
for item in date:
    if item in filenames:
        print(item)
    else:
        Mbox('Error', 'El archivo ' + item + ' no existe en el directorio actual:' + '\n' + str(path_input), 0x1000)
        print('El archivo ' + item + ' no se cargó correctamente')

Archivos proximos a cargar:
Mora_201505.txt
Mora_201504.txt
Mora_201503.txt
Mora_201502.txt
Mora_201501.txt
Mora_201412.txt
Mora_201411.txt
Mora_201410.txt
Mora_201409.txt
Mora_201408.txt
Mora_201407.txt
Mora_201406.txt
Mora_201405.txt
Mora_201404.txt
Mora_201403.txt
Mora_201402.txt
Mora_201401.txt


In [285]:
dataframes = []
for date in date:
    dataframes.append(pd.read_csv(date, sep=None, engine='python'))
for df in dataframes:
    df.drop(df.columns[[3,4]], axis=1, inplace=True)
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('@', '').str.replace('.', '').str.replace('á','a').str.replace('é','e').str.replace('í','i').str.replace('ó','o').str.replace('ú','u').str.replace(',','.')

In [286]:
num_mora_mes = []
i=0
for df in dataframes:
    df.dropna(thresh=1000 ,axis=1, inplace=True)
    num_mora_mes.append('mora' + str(i+2))
    df.columns = (['tipoid', 'id', 'obligacion', num_mora_mes[i]])
    i=i+1

In [287]:
#Creación del Tab_mora
i=0
tab_mora_final = tab_mora.copy()
for i in reversed(range(int(cant_mes_tab))):
    tab_mora_final = pd.merge(tab_mora_final, dataframes[i][['obligacion', num_mora_mes[i]]], on='obligacion', how='left')
cols = tab_mora_final.columns.tolist()
cols = cols[0:3] + cols[4:len(num_mora_mes)+4] + cols[3:4]
tab_mora_final = tab_mora_final[cols]
tab_mora_final2 = tab_mora_final.fillna(-1)
tab_mora_final2.head()

,id,obligacion,cod_credito,mora18,mora17,mora16,mora15,mora14,mora13,mora12,mora11,mora10,mora9,mora8,mora7,mora6,mora5,mora4,mora3,mora2,mora1
0,74300362,200000252372,2001,-1.0,0.0,0.0,3.0,0.0,1.0,0.0,1.0,3.0,0.0,1.0,4.0,11.0,4.0,4.0,3.0,1.0,0
1,87433209,200000019555,2001,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,4.0,0.0,1
2,19115492,200000467714,2001,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,30971295,200000014085,2001,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,2.0,4.0,0
4,19344384,200000099019,2001,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0


---

---

<h1><center><font size =7 color="#B3172D">Imputación</font> </center></h1>

---

---

### Realizado para toda la población

In [288]:
tab_mora_imputado = tab_mora_final2.copy()

In [289]:
range(int(cant_mes_tab)+1,2,-1)

range(18, 2, -1)

In [290]:
for pos in range(int(cant_mes_tab)+1,2,-1):
    tab_mora_imputado['mora'+str(pos-1)] = tab_mora_imputado.apply(lambda x: np.median([x['mora' + str(pos)], x['mora' + str(pos-2)]]) if abs(x['mora'+str(pos)]-x['mora'+str(pos-2)])<60 and
                                           x['mora'+str(pos)]!=-1 and x['mora'+str(pos-2)]!=-1 and x['mora'+str(pos-1)]==-1
                                           else (x['mora'+str(pos-1)]), axis=1)
    tab_mora_imputado['mora'+str(pos-1)] = tab_mora_imputado.apply(lambda x: x['mora'+str(pos)] if abs(x['mora'+str(pos)]-x['mora'+str(pos-2)])>60 and
                                           x['mora'+str(pos)]!=-1 and x['mora'+str(pos-2)]!=-1 and x['mora'+str(pos-1)]==-1
                                           else (x['mora'+str(pos-1)]), axis=1)
    tab_mora_imputado['mora'+str(pos-2)] = tab_mora_imputado.apply(lambda x: x['mora'+str(pos-1)] if 
                                           x['mora'+str(pos)]==-1 and x['mora'+str(pos-2)]==-1 and x['mora'+str(pos-1)]!=-1
                                           else (x['mora'+str(pos-2)]), axis=1)
    tab_mora_imputado['mora'+str(pos-2)] = tab_mora_imputado.apply(lambda x: x['mora'+str(pos-1)] if 
                                       x['mora'+str(pos)]!=-1 and x['mora'+str(pos-1)]!=-1 and x['mora'+str(pos-2)]==-1
                                       else x['mora'+str(pos-2)], axis=1)

In [291]:
tab_mora_imputado.head()

,id,obligacion,cod_credito,mora18,mora17,mora16,mora15,mora14,mora13,mora12,mora11,mora10,mora9,mora8,mora7,mora6,mora5,mora4,mora3,mora2,mora1
0,74300362,200000252372,2001,-1.0,0.0,0.0,3.0,0.0,1.0,0.0,1.0,3.0,0.0,1.0,4.0,11.0,4.0,4.0,3.0,1.0,0.0
1,87433209,200000019555,2001,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,4.0,0.0,1.0
2,19115492,200000467714,2001,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,30971295,200000014085,2001,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0
4,19344384,200000099019,2001,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [292]:
path_input=(r'\Users\migue\Python\behave_score\TAB_mora_Miguel')
os.chdir(r'\Users\migue\Python\behave_score\TAB_mora_Miguel')

In [293]:
#Guarda el TAB_MORA 
tab_mora_imputado.to_csv('TAB_mora_' + pob_objetivo + '.txt', header=True, index=False, sep='\t', mode='a')

In [294]:
#Comentar esta línea si no se tiene la libreria instalada
%notify
time.sleep(1)
end = time.time()
print(str(end - start) + ' s')

<IPython.core.display.Javascript object>

406.9958200454712 s


---

---

<h1><center><font size =7 color="#B3172D">Verificaciones</font> </center></h1>

<h2><center><font size =7 color="#094C73">Verificación antes de imputar valores</font> </center></h2>

---

---

In [ ]:
#BEHAVESCORE
path_input=(r'\Users\migue\Python\behave_score')
os.chdir(r'\Users\migue\Python\behave_score')
filenames_tab_mora=glob.glob('**/TAB_mora**.txt', recursive=True)

In [ ]:
#Leer el archivo de TAB_mora para comparar
file_name_tab_mora = ('TAB_mora_' + pob_objetivo + '.txt')
print(file_name_tab_mora)
if file_name_tab_mora in filenames_tab_mora:
    tab_mora_compare = pd.read_csv(file_name_mora1, sep=None, engine='python')
else:
    Mbox('Error', 'El archivo ' + file_name_tab_mora + ' no existe en el directorio actual:' + '\n' + str(path_input), 0x1000)

In [ ]:
tab_mora_compare_copy = tab_mora_compare.copy()
tab_mora_compare_sincorte = tab_mora_compare.copy()
tab_mora_compare_sincorte.drop('corte', axis=1, inplace=True)

In [ ]:
tab_mora_compare.sort_values('obligacion', ascending=True, inplace=True)
tab_mora_final2.sort_values('obligacion', ascending=True, inplace=True)
tab_mora_compare_copy.sort_values('obligacion', ascending=True, inplace=True)

In [ ]:
mora_compare=[]
for item in num_mora_mes:
    mora_compare.append(item)
mora_compare.insert(0,"mora1")
for i in mora_compare:                                                                              #1 comparación no coincide
    tab_mora_compare_copy[i+'_ver'] = np.where(tab_mora_compare_copy[i] == tab_mora_final2[i], 0, 1) #0 comparación correcta
tab_mora_compare_copy['marca']=1
columns = tab_mora_compare_copy.columns.tolist()
columns = columns[0:4] + columns[22:23]  + columns[-1:] + columns[23:41]  + columns [4:22] 
tab_mora_validacion = tab_mora_compare_copy[columns]
tab_mora_validacion.drop(tab_mora_validacion.columns[[24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41]], axis=1, inplace=True)
tab_mora_validacion.head()

In [ ]:
lista=['mora1_ver','mora2_ver','mora3_ver','mora4_ver','mora5_ver','mora6_ver','mora7_ver','mora8_ver','mora9_ver','mora10_ver','mora11_ver',
       'mora12_ver','mora13_ver','mora14_ver','mora17_ver','mora18_ver']
tab_mora_validacion.groupby(['marca'])[lista].agg('sum')

In [ ]:
tab_mora_validacion[tab_mora_validacion['mora5_ver']==1].head(10)

---

In [ ]:
display(tab_mora_final2.loc[tab_mora_final2['obligacion'] == 200000196540])
display(tab_mora_compare.loc[tab_mora_validacion['obligacion'] == 200000196540])

---

In [ ]:
display(tab_mora_final2.loc[tab_mora_final2['obligacion'] == 200000230983])
display(tab_mora_compare.loc[tab_mora_validacion['obligacion'] == 200000230983])

---

###### Inconsistencia entre el TAB_mora y el archivo mora 202002-202001-201912

In [ ]:
tab_mora_final2.loc[tab_mora_final2['obligacion'] == 22011042001598]

In [ ]:
tab_mora_compare.loc[tab_mora_validacion['obligacion'] == 22011042001598]

---

---

<h2><center><font size =7 color="#094C73">Verificación despúes de imputar valores</font> </center></h2>

---

---

In [ ]:
display(tab_mora_compare_copy.head())
display(tab_mora_compare_copy.shape)
display(tab_mora_imputado.head())
display(tab_mora_imputado.shape)

In [ ]:
display(type(tab_mora_compare_copy[i])) #== tab_mora_imputado[i]
display(type(tab_mora_imputado[i]))

In [ ]:
tab_mora_compare_imputado_validacion = tab_mora_imputado.copy()
for i in mora_compare:
                                                                                                                        #1 comparación no coincide
    tab_mora_compare_imputado_validacion[i+'_ver'] = np.where(tab_mora_compare_copy[i] == tab_mora_imputado[i], 0, 1) #0 comparación correcta
tab_mora_compare_imputado_validacion['marca']=1
columnas = tab_mora_compare_imputado_validacion.columns.tolist()
columnas = columnas[0:3] + columnas[21:39] + columnas[3:21] + columnas[-1:]
tab_mora_compare_imputado_validacion2 = tab_mora_compare_imputado_validacion[columnas]
tab_mora_compare_imputado_validacion2.drop(tab_mora_compare_imputado_validacion2.columns[[21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38]], axis=1, inplace=True)
tab_mora_compare_imputado_validacion2.head()

In [ ]:
lista=['mora1_ver','mora2_ver','mora3_ver','mora4_ver','mora5_ver','mora6_ver','mora7_ver','mora8_ver','mora9_ver','mora10_ver','mora11_ver',
       'mora12_ver','mora13_ver','mora14_ver','mora17_ver','mora18_ver']
tab_mora_validacion.groupby(['marca'])[lista].agg('sum')

In [ ]:
tab_mora_compare_imputado_validacion2.groupby(['marca'])[lista].agg('sum')

In [ ]:
tab_mora_compare_imputado_validacion2[tab_mora_compare_imputado_validacion2['mora6_ver']==1].head(10)

In [ ]:
#Muestra la mora de cada me dentro del vector por separado de un crédito
for i in np.arange(0,17,1):
    display(dataframes[i].loc[dataframes[i]['obligacion']==220002833887])

In [ ]:
#Créditos los cuales la mora18 y la mora 1 es diferente de -1, por lo tanto, no tienen -1 dentro del vector
creditos_vigentes = tab_mora_imputado.loc[(tab_mora_imputado['mora18'] != -1) & (tab_mora_imputado['mora1'] !=-1)]
print('Cantidad de créditos sin ningún valor -1 en el vector construido del TAB_mora es: ' + str(creditos_vigentes.shape[0]) + ' de un total de: ' + str(tab_mora_imputado.shape[0]) + '\n')
print('Equivalente a un: ' + str(round(creditos_vigentes.shape[0]/tab_mora_imputado.shape[0]*100)) + '% de los créditos' + '\n')
print('Porcentaje de créditos que presentan error luego de las imputaciones: '+ str(round(7197/tab_mora_imputado.shape[0]*100)) + '%'+ ('\n')*2)
print('Créditos los cuales la mora18 y la mora 1 es diferente de -1, por lo tanto, no tienen -1 dentro del vector'+'\n')
for i in mora_compare:
    print('La cantidad de clientes en el TAB_mora luego de las imputaciones, con valor -1 en ' + i + ' es igual a: ' + str((creditos_vigentes[i].values == -1).sum()) + '\n')

---

---

### Pruebas con clientes que no coinciden en mora2
#### en mora 1 ambos dataframes son iguales

In [ ]:
display(tab_mora_compare_sincorte.loc[tab_mora_compare_sincorte['obligacion'] == 200000196540])
display(tab_mora_final2.loc[tab_mora_final2['obligacion'] == 200000196540])
display(tab_mora_imputado.loc[tab_mora_imputado['obligacion'] == 200000196540])

In [ ]:
display(tab_mora_compare_sincorte.loc[tab_mora_compare_sincorte['obligacion'] == 210000086584])
display(tab_mora_final2.loc[tab_mora_final2['obligacion'] == 210000086584])
display(tab_mora_imputado.loc[tab_mora_imputado['obligacion'] == 210000086584])

In [ ]:
display(tab_mora_compare_sincorte.loc[tab_mora_compare_sincorte['obligacion'] == 210000092572])
display(tab_mora_final2.loc[tab_mora_final2['obligacion'] == 210000092572])
display(tab_mora_imputado.loc[tab_mora_imputado['obligacion'] == 210000092572])

In [ ]:
display(tab_mora_compare_sincorte.loc[tab_mora_compare_sincorte['obligacion'] == 210000189189])
display(tab_mora_final2.loc[tab_mora_final2['obligacion'] == 210000189189])
display(tab_mora_imputado.loc[tab_mora_imputado['obligacion'] == 210000189189])

In [ ]:
display(tab_mora_compare_sincorte.loc[tab_mora_compare_sincorte['obligacion'] == 220001614062])
display(tab_mora_final2.loc[tab_mora_final2['obligacion'] == 220001614062])
display(tab_mora_imputado.loc[tab_mora_imputado['obligacion'] == 220001614062])

In [ ]:
display(tab_mora_compare_sincorte.loc[tab_mora_compare_sincorte['obligacion'] == 220001988554])
display(tab_mora_final2.loc[tab_mora_final2['obligacion'] == 220001988554])
display(tab_mora_imputado.loc[tab_mora_imputado['obligacion'] == 220001988554])

In [ ]:
display(tab_mora_compare_sincorte.loc[tab_mora_compare_sincorte['obligacion'] == 220002248448])
display(tab_mora_final2.loc[tab_mora_final2['obligacion'] == 220002248448])
display(tab_mora_imputado.loc[tab_mora_imputado['obligacion'] == 220002248448])

In [ ]:
display(tab_mora_compare_sincorte.loc[tab_mora_compare_sincorte['obligacion'] == 220002833887])
display(tab_mora_final2.loc[tab_mora_final2['obligacion'] == 220002833887])
display(tab_mora_imputado.loc[tab_mora_imputado['obligacion'] == 220002833887])

In [ ]:
display(tab_mora_compare_sincorte.loc[tab_mora_compare_sincorte['obligacion'] == 210000189189])
display(tab_mora_final2.loc[tab_mora_final2['obligacion'] == 210000189189])
display(tab_mora_imputado.loc[tab_mora_imputado['obligacion'] == 210000189189])

### Pruebas con clientes que no coinciden en mora18

In [ ]:
#CEDULAS
numero = 0
cedulas = [131688,200000004329,200000034874, 200000798584,220002833887,
           200000798600,200000784328,200000784239,200000784060,200000783536,
           220002833887,210000189189,220001988554,210000189189,220002248448]
for i in cedulas:
    if numero < 5:
        print('Franklin')
        numero = numero +1
    elif numero<10:
        print('Zuleyma')
        numero = numero +1
    elif numero<15:
        print('Jennyfer')
        numero = numero +1 
    print('Número del crédito: ' + str(i) + '\n')
    print('TAB_mora homologado')
    display(tab_mora_compare_sincorte.loc[tab_mora_compare_sincorte['obligacion'] == i])
    print('\n')
    print('TAB_mora imputado')
    display(tab_mora_imputado.loc[tab_mora_imputado['obligacion'] == i])
    print('\n')
    print('-------------------------------------------------------------------------------------------------------------------------------')
    print('\n')

In [ ]:
display(tab_mora_compare_sincorte.loc[tab_mora_compare_sincorte['obligacion'] == 131688])
display(tab_mora_final2.loc[tab_mora_final2['obligacion'] == 131688])
display(tab_mora_imputado.loc[tab_mora_imputado['obligacion'] == 131688])

In [ ]:
display(tab_mora_compare_sincorte.loc[tab_mora_compare_sincorte['obligacion'] == 200000004329])
display(tab_mora_final2.loc[tab_mora_final2['obligacion'] == 200000004329])
display(tab_mora_imputado.loc[tab_mora_imputado['obligacion'] == 200000004329])

In [ ]:
display(tab_mora_compare_sincorte.loc[tab_mora_compare_sincorte['obligacion'] == 200000007026])
display(tab_mora_final2.loc[tab_mora_final2['obligacion'] == 200000007026])
display(tab_mora_imputado.loc[tab_mora_imputado['obligacion'] == 200000007026])

In [ ]:
display(tab_mora_compare_sincorte.loc[tab_mora_compare_sincorte['obligacion'] == 200000008159])
display(tab_mora_final2.loc[tab_mora_final2['obligacion'] == 200000008159])
display(tab_mora_imputado.loc[tab_mora_imputado['obligacion'] == 200000008159])

In [ ]:
display(tab_mora_compare_sincorte.loc[tab_mora_compare_sincorte['obligacion'] == 200000011501])
display(tab_mora_final2.loc[tab_mora_final2['obligacion'] == 200000011501])
display(tab_mora_imputado.loc[tab_mora_imputado['obligacion'] == 200000011501])

<h4><center><font size =5 color="#094C73">FIN</font></center> </h4>

---

---